In [28]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict_booster, load_model, Metrics, predict_xgbm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# process_train_data()
# process_test_data()

train_data, test_data = load_split_processed_data()

model_name = "BEST_LightGbmV2_p069_r033_090auc_90_features.pickle"

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

None

-----------Train----------

Metrics
AUC: 0.96
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    140413
           1       0.83      0.42      0.56      9585

    accuracy                           0.96    149998
   macro avg       0.89      0.71      0.77    149998
weighted avg       0.95      0.96      0.95    149998

-----------Test----------

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.69      0.33      0.45      9403

    accuracy                           0.95    150000
   macro avg       0.82      0.66      0.71    150000
weighted avg       0.94      0.95      0.94    150000

